In [0]:
%run ./0.Config

In [0]:
def send_vendor_emails_func(_: str) -> str:
    """
    Sends customized emails to shortlisted and non-selected vendors
    based on their RFQ evaluation status. Pulls data from Delta Lake,
    generates personalized messages with a Databricks LLM, and sends emails via SMTP.
    """
    shortlisted_vendors = spark.read.option("header", "true").csv(
        "abfss://vendor-rfq@genaiautomationsa.dfs.core.windows.net/processed/Shortlisted_vendors")
    vendors_data = spark.read.option("header", "true").csv(
        "abfss://vendor-rfq@genaiautomationsa.dfs.core.windows.net/vendor_data/vendor_data.csv")

    shortlisted_vendors_df = shortlisted_vendors.toPandas()
    vendors_data_df = vendors_data.toPandas()

    shortlisted_emails = shortlisted_vendors_df["Vendor Email"].tolist()
    non_selected_vendors_df = vendors_data_df[~vendors_data_df["Vendor Email"].isin(shortlisted_emails)]

    llm = ChatDatabricks(endpoint="databricks-llama-4-maverick")

    email_prompt = """
    You are a professional Procurement Manager replying to a vendor about their RFQ submission.
    Vendor Name: {vendor_name}
    Status: {status}
    Routes: {routes}

    If Shortlisted:
    - Congratulate the vendor.
    - Inform them they are partially selected for the procurement program.
    - Let them know our procurement manager will contact them soon.

    If Not Selected:
    - Thank them for their submission.
    - Politely inform them they weren’t selected due to pricing or other criteria.
    - Encourage them to participate in future RFQs.

    Sign off with:
    Best regards,  
    Coca Cola Pvt Ltd  
    procurement@cocacola.com
    """

    grouped = shortlisted_vendors_df.groupby("Vendor Email").agg({
        "Vendor Name": "first",
        "Route ID": lambda x: ', '.join(sorted(set(x)))
    }).reset_index()

    yag = yagmail.SMTP(EMAIL_USER, EMAIL_PASSWORD)

    for _, vendor in grouped.iterrows():
        content = llm.invoke(email_prompt.format(
            vendor_name=vendor["Vendor Name"], status="Shortlisted", routes=vendor["Route ID"]
        )).content
        yag.send(to=vendor["Vendor Email"], subject="You're Shortlisted", contents=content)
        print(f"✅ Email sent to shortlisted vendor: {vendor['Vendor Name']}")

    for _, vendor in non_selected_vendors_df.iterrows():
        content = llm.invoke(email_prompt.format(
            vendor_name=vendor["Vendor Name"], status="Not Selected", routes="None"
        )).content
        yag.send(to=vendor["Vendor Email"], subject="Thank You", contents=content)
        print(f"✅ Email sent to non-selected vendor: {vendor['Vendor Name']}")

    return "📨 Emails sent to shortlisted and non-selected vendors successfully."

# Wrap the function as a LangChain Tool
send_vendor_emails_tool = Tool(
    name="send_vendor_emails",
    func=send_vendor_emails_func,
    description="Sends personalized emails to shortlisted and non-selected vendors based on evaluation results.",
    return_direct=True
)


In [0]:
tools = [send_vendor_emails_tool]

agent = initialize_agent(
    tools=tools,
    llm=ChatDatabricks(endpoint="databricks-llama-4-maverick"),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("Sends personalized emails to shortlisted and non-selected vendors based on evaluation results")